In [20]:
import torch
import numpy as np
from glob import glob


In [22]:
files = np.array(glob("data/*/*/*"))
print('There are %d total images.' % len(files))

There are 1057 total images.


In [23]:
import os
from torchvision import datasets

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import torchvision.transforms as transforms


from torch.utils.data.sampler import SubsetRandomSampler

In [26]:
num_workers = 0
batch_size = 32

transform_train = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),
                                      transforms.RandomRotation(20),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

transform_test = transforms.Compose([transforms.Resize(224),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])])

data_dir = 'data'

#loading images in train validate and test folders into dictionary loaders_scratch with 
# train, valid, test as keys

train_data = datasets.ImageFolder(os.path.join(data_dir, 'train/'), transform=transform_train)
test_data = datasets.ImageFolder(os.path.join(data_dir, 'test/'), transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=False)

loaders_scratch = {'train': train_loader,
                   'test': test_loader}

#classes = ['apple', 'orange', 'banana', 'cardboard', 'bottle', 'can', 'mixed']

In [27]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
        ## Define layers of a CNN
        
        '''using stride 2 in place of maxpooling layers'''
        
        # in : 3*224*224
        self.conv1 = nn.Conv2d(3, 16, 4, stride = 2, padding = 1)
        
        # in :16* 112 *112
        self.conv2 = nn.Conv2d(16, 32, 4, stride = 2, padding = 1)
        
        # in : 32 * 56 * 56
        self.conv3 = nn.Conv2d(32, 64, 4, stride = 2, padding = 1)
        # out : 64*28*28
        
        self.fc1 = nn.Linear(64*7*7, 784)
        self.fc2 = nn.Linear(784, 133)
        
        self.dropout = nn.Dropout(p = 0.2)
        self.batch_norm1 = nn.BatchNorm2d(32)
        self.batch_norm2 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(2, 2)
    
    def forward(self, x):
        ## Define forward behavior
        
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.batch_norm1(self.conv2(x)), 0.2)
        x = self.maxpool(x)
        x = F.leaky_relu(self.batch_norm2(self.conv3(x)), 0.2)
        x = self.maxpool(x)
        
        x = x.view(-1, 64*7*7)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        
        # No relu here, we use CELoss
        x = self.fc2(x)
        
        
        
        return x

#-#-# You so NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model_scratch = Net()

In [28]:
import torch.optim as optim

### TODO: select loss function
criterion_scratch = nn.CrossEntropyLoss()

### TODO: select optimizer
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr = 0.006)

In [33]:
def train(n_epochs, loaders, model, optimizer, criterion, save_path):
    """returns trained model"""
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
            optimizer.zero_grad()
            
            #typical train step:
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))

            
        # print training statistics 
        print('Epoch: {} \tTraining Loss: {:.6f}'.format(
            epoch, 
            train_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        
        torch.save(model.state_dict(), save_path) 
    # return trained model
    return model

In [34]:
model_scratch = train(40, loaders_scratch, model_scratch, optimizer_scratch, 
                      criterion_scratch, 'model_scratch.pt')

C:\Users\User\Anaconda3\lib\site-packages\PIL\Image.py:968: UserWarning: Palette images with Transparency   expressed in bytes should be converted to RGBA images
  ' expressed in bytes should be converted ' +


Epoch: 1 	Training Loss: 2.894610
Epoch: 2 	Training Loss: 1.513670
Epoch: 3 	Training Loss: 1.261914
Epoch: 4 	Training Loss: 1.094280
Epoch: 5 	Training Loss: 0.958900
Epoch: 6 	Training Loss: 0.873369
Epoch: 7 	Training Loss: 0.795217
Epoch: 8 	Training Loss: 0.752070
Epoch: 9 	Training Loss: 0.702412
Epoch: 10 	Training Loss: 0.640562
Epoch: 11 	Training Loss: 0.644403
Epoch: 12 	Training Loss: 0.605139
Epoch: 13 	Training Loss: 0.583485
Epoch: 14 	Training Loss: 0.566044
Epoch: 15 	Training Loss: 0.557963
Epoch: 16 	Training Loss: 0.555721
Epoch: 17 	Training Loss: 0.512514
Epoch: 18 	Training Loss: 0.521322
Epoch: 19 	Training Loss: 0.483293
Epoch: 20 	Training Loss: 0.485182
Epoch: 21 	Training Loss: 0.452940
Epoch: 22 	Training Loss: 0.469876
Epoch: 23 	Training Loss: 0.468967
Epoch: 24 	Training Loss: 0.471820
Epoch: 25 	Training Loss: 0.445160
Epoch: 26 	Training Loss: 0.423988
Epoch: 27 	Training Loss: 0.423210
Epoch: 28 	Training Loss: 0.437956
Epoch: 29 	Training Loss: 0.3

In [35]:
def test(loaders, model, criterion):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_scratch, model_scratch, criterion_scratch)

Test Loss: 0.691966


Test Accuracy: 76% (156/204)


## Trasfer Learning

In [36]:
loaders_transfer = loaders_scratch

In [37]:
import torchvision.models as models
import torch.nn as nn


model_transfer = models.resnet50(pretrained = True)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to C:\Users\User/.cache\torch\checkpoints\resnet50-19c8e357.pth
100%|██████████████████████████████████████████████████████████████| 102502400/102502400 [00:09<00:00, 11287050.23it/s]


In [38]:
model_transfer

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [39]:
#freeze pretrained model parameters
for param in model_transfer.parameters():
    param.requires_grad = False
    
# resnet classifies 1000 categories, but we want only 133:

model_transfer.fc = nn.Linear(2048, 133)


In [40]:
criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.Adam(model_transfer.fc.parameters(), lr = 0.01)

In [ ]:
# train the model
model_transfer = train(10, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, 'model_transfer.pt')

# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

Epoch: 1 	Training Loss: 1.750549
Epoch: 2 	Training Loss: 0.333977
Epoch: 3 	Training Loss: 0.122336
Epoch: 4 	Training Loss: 0.072873
Epoch: 5 	Training Loss: 0.121644


In [ ]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)